In [1]:
import os
import sys
sys.path.insert(0, "/Users/edwardmorgan/acados/interfaces/acados_template")
import shutil
from pathlib import Path

import casadi as ca
import numpy as np
from acados_template import AcadosModel, AcadosOcp, AcadosOcpSolver

In [2]:
# ───────────────────── CONFIG ─────────────────────
N_HORIZON = 20
DT = 0.01
G_VAL = 0.0
N_JOINTS = 4
N_THETA = 8 * N_JOINTS

LOWER_JOINT_LIMIT = np.array([1, 0.01, 0.01, 0.01])
UPPER_JOINT_LIMIT = np.array([5.50, 3.40, 3.40, 5.70])
U_MIN_ARM = np.array([-3.83664, -1.629139, -1.518764, -0.54])
U_MAX_ARM = np.array([3.83664, 1.629139, 1.518764, 0.54])

PAYLOAD_PROPS = np.zeros(4)
EPS_TORQUE = 0.1 * np.ones(N_JOINTS)

sigma_theta = 0.005
QY = 1e3 * np.ones(N_JOINTS)  # velocity residual weight
R_C = 1e-2 * np.ones(N_JOINTS)  # torque residual weight
R_W = (1 / sigma_theta**2) * np.ones(N_THETA)
R_U = np.concatenate([R_C, R_W])

try:
    THIS_DIR = Path(__file__).resolve().parent
except NameError:
    THIS_DIR = Path.cwd()

CASADI_FN = THIS_DIR / "arm.casadi"
LIB_NAME = THIS_DIR / "libMHEarm.so"


x0_bar = np.array([2, 1.5, 1.5, 2.5, 0, 0, 0, 0])
# Pre‑identified parameter vector θ₀ (32 values)
THETA0 = np.concatenate([
    np.array([
        1.23820451e02, 1.23820437e02, 1.23820199e02, 1.23820446e02,
        1.23818582e02, 1.23820445e02, 1.23820450e02, 1.23820451e02,
    ]),
    np.array([
        2.65212180e00, 2.23801687e00, 1.36440968e00, 3.53203598e-01,
        3.09552577e00, 3.52105211e00, 9.76417047e-01, 3.57275142e-01,
    ]),
    np.array([
        1.71638216e-17, 7.61787573e-16, 8.95118893e-02, 2.56682942e-18,
        1.94048838e-17, 2.00526572e-02, 1.61355205e-16, 4.18920129e-03,
    ]),
    np.array([
        3.97767623e-01, 1.74679387e-01, 1.31099819e-01, 5.15490167e-05,
        1.20981900e-13, 7.12518262e-02, 3.49942730e-02, 3.90392008e-02,
    ]),
])

# ───────────────── MODEL ─────────────────

def build_augmented_model() -> AcadosModel:
    if not CASADI_FN.exists():
        raise FileNotFoundError("arm.casadi missing — generate it first.")

    F_next = ca.Function.load(str(CASADI_FN)).expand()

    x_mech = ca.SX.sym("x_mech", 2 * N_JOINTS)
    theta = ca.SX.sym("theta", N_THETA)
    w_theta = ca.SX.sym("w_theta", N_THETA)
    u_tau = ca.SX.sym("u", N_JOINTS)

    x_next_mech = F_next(
        x_mech,
        u_tau,
        DT,
        G_VAL,
        PAYLOAD_PROPS,
        theta,
        LOWER_JOINT_LIMIT,
        UPPER_JOINT_LIMIT,
        EPS_TORQUE,
    )

    x_next = ca.vertcat(x_next_mech, theta + w_theta)
    x_aug = ca.vertcat(x_mech, theta)
    u_aug = ca.vertcat(u_tau, w_theta)

    mdl = AcadosModel()
    mdl.name = "arm_mhe"
    mdl.x = x_aug
    mdl.u = u_aug
    mdl.disc_dyn_expr = x_next
    return mdl


In [6]:
# ───────────────── OCP BUILDER ────────────────

def export_mhe_solver(N: int = N_HORIZON) -> AcadosOcpSolver:
    model = build_augmented_model()
    nx = model.x.size()[0]
    nu = model.u.size()[0]

    ny_vel = N_JOINTS
    ny_total = ny_vel + nu

    ocp = AcadosOcp()
    ocp.model = model

    ocp.dims.ny = ny_total
    ocp.dims.ny_e = ny_vel
    ocp.dims.ny_0 = 0

    # cost LS matrices
    Vx = np.zeros((ny_total, nx))
    Vx[:ny_vel, N_JOINTS : 2 * N_JOINTS] = np.eye(ny_vel)

    Vu = np.zeros((ny_total, nu))
    Vu[ny_vel : ny_vel + N_JOINTS, :N_JOINTS] = np.eye(N_JOINTS)
    Vu[ny_vel + N_JOINTS :, N_JOINTS:] = np.eye(N_THETA)

    ocp.cost.cost_type = "LINEAR_LS"
    ocp.cost.cost_type_e = "LINEAR_LS"
    ocp.cost.Vx = Vx
    ocp.cost.Vu = Vu
    ocp.cost.W = np.diag(np.concatenate([QY, R_U]))
    ocp.cost.yref = np.zeros(ny_total)

    Vx_e = np.zeros((ny_vel, nx))
    Vx_e[:, N_JOINTS : 2 * N_JOINTS] = np.eye(ny_vel)

    ocp.cost.Vx_e = Vx_e
    ocp.cost.W_e = np.diag(QY)
    ocp.cost.yref_e = np.zeros(ny_vel)

    ocp.constraints.x0 = np.concatenate([x0_bar, THETA0])
    ocp.constraints.lbu   = U_MIN_ARM                 # length 4
    ocp.constraints.ubu   = U_MAX_ARM                 # length 4
    ocp.constraints.idxbu = np.arange(N_JOINTS)       # [0 1 2 3]

    opts = ocp.solver_options
    
    opts.N_horizon = N
    opts.integrator_type = "DISCRETE"
    # opts.qp_solver = "FULL_CONDENSING_QPOASES"
    opts.qp_solver = "PARTIAL_CONDENSING_HPIPM"
    opts.qp_solver_cond_N = 5        # trade‑off: less code, similar runtime
    opts.hessian_approx = "GAUSS_NEWTON"
    opts.nlp_solver_type = "SQP_RTI"
    opts.print_level = 0
    opts.tf = N * DT
    ocp.solver_options.sens_algebraic = 0
    ocp.solver_options.sens_u         = 0
    ocp.solver_options.sens_x         = 0

    print("\n⏳  Generating and compiling MHE solver…")

    json_path = Path("arm_mhe.json")
    lib_path  = Path("libacados_ocp_solver_arm_mhe.so")

    solver = AcadosOcpSolver(
        ocp,
        json_file=str(json_path),
        generate=not json_path.exists(),   # ⇐ generate on first run
        build=not lib_path.exists(),       # ⇐ compile on first run
    )
    try:
        shutil.copy(Path(solver.shared_lib_name), LIB_NAME)
        print(f"✔  Shared library saved to {LIB_NAME}")
    except Exception as exc:
        print(f"⚠  Could not copy shared lib: {exc}")

    return solver

In [7]:
# ───────────────── RUNTIME STEP ───────────────

def mhe_step(solver: AcadosOcpSolver, vel_meas: np.ndarray, tau_meas: np.ndarray):
    if vel_meas.shape != (N_JOINTS,) or tau_meas.shape != (N_JOINTS,):
        raise ValueError("Shape mismatch")

    solver.shift_states()
    solver.shift_inputs()

    yref = np.concatenate([vel_meas, tau_meas, np.zeros(N_THETA)])
    u_aug = np.concatenate([tau_meas, np.zeros(N_THETA)])

    for k in range(solver.acados_ocp.dims.N):
        solver.set(k, "y_ref", yref)
        solver.set(k, "u", u_aug)

    solver.set(solver.acados_ocp.dims.N, "y_ref_e", vel_meas)

    if solver.solve() != 0:
        raise RuntimeError("Solver failed")

    return solver.get(0, "x")[-N_THETA:]


In [ ]:
# ───────────────── DEMO ───────────────────────

solver = export_mhe_solver()

# vel_meas = np.zeros(N_JOINTS)  # replace with actual velocity measurements
# tau_meas = np.zeros(N_JOINTS)  # replace with actual torque measurements

# print("Smoke‑test loop: printing θ̂ for five iterations")
# for k in range(5):
#     theta_hat = mhe_step(solver, vel_meas, tau_meas)
#     print(f"step {k}: θ̂ = {theta_hat}")


⏳  Generating and compiling MHE solver…
rm -f libacados_ocp_solver_arm_mhe.dylib
rm -f acados_solver_arm_mhe.o
cc -fPIC -std=c99   -O2 -I/Users/edwardmorgan/acados/include -I/Users/edwardmorgan/acados/include/acados -I/Users/edwardmorgan/acados/include/blasfeo/include -I/Users/edwardmorgan/acados/include/hpipm/include  -c -o acados_solver_arm_mhe.o acados_solver_arm_mhe.c
cc -fPIC -std=c99   -O2 -I/Users/edwardmorgan/acados/include -I/Users/edwardmorgan/acados/include/acados -I/Users/edwardmorgan/acados/include/blasfeo/include -I/Users/edwardmorgan/acados/include/hpipm/include  -c -o arm_mhe_model/arm_mhe_dyn_disc_phi_fun.o arm_mhe_model/arm_mhe_dyn_disc_phi_fun.c
cc -fPIC -std=c99   -O2 -I/Users/edwardmorgan/acados/include -I/Users/edwardmorgan/acados/include/acados -I/Users/edwardmorgan/acados/include/blasfeo/include -I/Users/edwardmorgan/acados/include/hpipm/include  -c -o arm_mhe_model/arm_mhe_dyn_disc_phi_fun_jac.o arm_mhe_model/arm_mhe_dyn_disc_phi_fun_jac.c
cc -fPIC -std=c99   